https://mgimond.github.io/Spatial/spatial-autocorrelation.html

https://www.fes.de/koordinatensysteme -> benachbarte Koordinaten +/- 0.1 (ca. 10km)

In [1]:
import pandas as pd
import numpy as np

words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

In [2]:
for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep = "\t")
    df = df.drop(columns = ["Unnamed: 0", "Fragebogen-Nr. Maurer", "Fragebogen-Nr. DWA", "Item Maurer", "Item DWA", "z_score_item", "Phontype Maurer", "Phontype DWA", "z_score_phontype", "Lextype Maurer", "Lextype DWA", "z_score_lextype"])
    df_item = df.drop(columns = ["Levenshtein Phontype Min", "Levenshtein Lextype Min"])
    df_item = df_item.drop(df[df["Levenshtein Item Min"] == 999].index)
    df_item = df_item.reset_index(drop=True)

    liste = list(df_item["Levenshtein Item Min"])
    mittel = np.mean(liste)
    stvar = np.std(liste)
    df_item["z_score"] = ((df_item["Levenshtein Item Min"]-mittel)/stvar)

    df_item_t = df_item.T

    array_list = []
    for id_1 in df_item_t:
        liste = []
        for id_2 in df_item_t:
            if id_1 != id_2 and (df_item_t[id_1][1] + 0.1) >= df_item_t[id_2][1] and (df_item_t[id_1][1] - 0.1) <= df_item_t[id_2][1] and (df_item_t[id_1][2] + 0.1) >= df_item_t[id_2][2] and (df_item_t[id_1][2] - 0.1) <= df_item_t[id_2][2]:
                liste.append(1)
            else:
                liste.append(0)
        array_list.append(liste)
    array = np.array(array_list)
    array_t = array.transpose()

    liste_array_new = []
    for array in array_t:
        array_n = [i for i in array if i != 0]
        length = len(array_n)
        if length == 0:
            mittel = 0
        else:
            mittel = 1/length
        array2 = [mittel if x!=0 else x for x in array]
        liste_array_new.append(array2)
    array_new = np.array(liste_array_new)
    array_new_t = array_new.transpose()

    liste_moran = []
    index_liste = 0
    for liste in array_new_t:
        index = 0
        m = 0
        for element in liste:
            prod = element * df_item_t[index][4]
            m += prod
            index += 1
        moran = m * df_item_t[index_liste][4]
        liste_moran.append(moran)
        index_liste += 1

    df_item_2 = df_item_t.T
    df_item_2["morans_i"] = liste_moran
    df_item_2.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/local_morans_i/moran_{word}.csv", sep = "\t", index = False)